In [2]:
import glob
import os
import pandas as pd
import numpy as np
import math
import re

In [62]:
os.chdir('C:\\Users\\Admin\\Desktop\\수업자료\\산학프로젝트\\fit_model\\blob-quickstart-v12\\학습데이터0624\\0624체조\\test')
os.getcwd()


'C:\\Users\\Admin\\Desktop\\수업자료\\산학프로젝트\\fit_model\\blob-quickstart-v12\\학습데이터0624\\0624체조\\test'

In [64]:
acc_list = glob.glob(os.path.join('./*ACCELEROMETER.csv'))
gy_list = glob.glob(os.path.join('./*GYROSCOPE.csv'))
# print(acc_list)
# print()
# print(gy_list)

In [65]:
all_data = []
idx = 0
for i in range(len(acc_list)):
    exercise_list = []
    s = acc_list[i]
    s = re.findall('\(([^)]+)',s)
    df_acc = pd.read_csv(acc_list[i])
    df_acc.insert( 0, 'id', None)
    df_gyro = pd.read_csv(gy_list[i])
    temp = len(df_acc)-len(df_gyro)
    if len(df_acc) > len(df_gyro):
        df_acc = df_acc[:len(df_acc)-temp]
    elif len(df_acc) < len(df_gyro):
        df_gyro = df_gyro[:len(df_gyro)+temp]
    df_acc=df_acc.drop('TimeZone',axis=1)
    df_acc.rename(columns = {'X' : 'acc_x'}, inplace = True)
    df_acc.rename(columns = {'Y' : 'acc_y'}, inplace = True)
    df_acc.rename(columns = {'Z' : 'acc_z'}, inplace = True)
    df_acc['acc_x'] = df_acc['acc_x'] / 9.8
    df_acc['acc_y'] = df_acc['acc_y'] / 9.8
    df_acc['acc_z'] = df_acc['acc_z'] / 9.8 

    df_gyro=df_gyro.drop('TimeZone',axis=1)
    df_gyro.rename(columns = {'X' : 'gy_x'}, inplace = True)
    df_gyro.rename(columns = {'Y' : 'gy_y'}, inplace = True)
    df_gyro.rename(columns = {'Z' : 'gy_z'}, inplace = True)
    df_acc['gy_x'] = df_gyro['gy_x']
    df_acc['gy_y'] = df_gyro['gy_y']
    df_acc['gy_z'] = df_gyro['gy_z']
    id_list = []
    count = 0
    exercise = acc_list[i]
    if len(df_acc)>3000:
        df_acc = df_acc[:3000]
    elif len(df_acc)>2400:
        df_acc = df_acc[:2400]
    elif len(df_acc)>1800:
        df_acc = df_acc[:1800]
    for i in range(len(df_acc)):
        if count>599:
            count = 0
            idx+=1
        id_list.append(idx)
        exercise_list.extend(s)
        count+=1
        if i == len(df_acc)-1:
            idx+=1
    df_acc['id'] = id_list
    df_acc['exercise'] = exercise_list
    all_data.append(df_acc)
df_all = pd.concat(all_data, axis=0, ignore_index=True)

In [66]:
df_all

,id,Time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z,exercise
0,0,2021-06-21 16:49:00:79942,0.832116,-0.407268,-0.417767,-5.46,45.43,-26.95,두손
1,0,2021-06-21 16:49:00:8194,0.846766,-0.392374,-0.405315,-3.43,5.74,1.05,두손
2,0,2021-06-21 16:49:00:83957,0.846034,-0.427046,-0.405071,-2.24,4.27,0.49,두손
3,0,2021-06-21 16:49:00:86032,0.848231,-0.417767,-0.415814,-0.49,3.99,1.05,두손
4,0,2021-06-21 16:49:00:87975,0.853359,-0.438766,-0.405071,4.90,4.13,1.54,두손
...,...,...,...,...,...,...,...,...,...
11995,19,2021-06-21 17:04:48:62764,0.868985,-0.391153,-0.276640,-3.71,4.34,-7.98,왼손
11996,19,2021-06-21 17:04:48:64842,0.874357,-0.371376,-0.279081,-7.21,3.01,-9.17,왼손
11997,19,2021-06-21 17:04:48:66756,0.895111,-0.385049,-0.280790,-7.84,-0.28,-10.71,왼손
11998,19,2021-06-21 17:04:48:68735,0.906098,-0.389444,-0.286895,-8.33,-2.31,-11.41,왼손


In [67]:
df_all.to_csv('./0624test.csv',encoding='cp949',index=False)

In [38]:
df_all.drop(['Time','acc_x','acc_y','acc_z','gy_x','gy_y','gy_z'],axis=1,inplace=True)

In [40]:
df_all
# 박수 0
# 두손 1
# 왼손 2
# 오른손 3

In [42]:
df_all.insert(1,'label',0)

In [ ]:
dic = {"박수":0,"두손":1,"왼손":2,"오른손":3}


In [49]:
a = list(df_all['exercise'])
for i in range(len(a)):
    if a[i] in dic:
        a[i] = dic[a[i]]
df_all['label'] = a

In [57]:
df_all.drop_duplicates(inplace=True)

In [55]:
df_all.rename(columns={'exercise':'label_desc'},inplace=True)

In [60]:
df_all.to_csv('./train_labels.csv',index=False,encoding='cp949')